# Combining the low mass and high mass x-ray binary data sets

For now we have the data sets hmxbcat which has only high mass xray binaries, and lmxbcat that only has low mass binaries. We want to create a training set and a testing set. Start by opening the fits files and extracting the repective tables. I split the tables in two marking a middle index point.



In [1]:
from astropy.io import fits
from astropy.table import Table, vstack
import os

In [2]:
home = os.path.expanduser('~')
workdir = home + "/XBinary-Classifier"

In [3]:
lmxbcat_file = workdir + "/lmxbcat.fits"
hmxbcat_file = workdir + "/hmxbcat.fits"

In [4]:
hdul_lmxb = fits.open(lmxbcat_file)
hdul_hmxb = fits.open(hmxbcat_file)

In [5]:
table_lmxb = Table(hdul_lmxb[1].data)
table_hmxb = Table(hdul_hmxb[1].data)

In [6]:
lmxb_mid_index = len(table_lmxb) // 2
hmxb_mid_index = len(table_hmxb) // 2

* Training set

The first halves of the both tables are combined together, and adding a target column

In [7]:
lmxb_first_half = table_lmxb[:lmxb_mid_index]
hmxb_first_half = table_hmxb[:hmxb_mid_index]

In [8]:
lmxb_first_half['target'] = 0
hmxb_first_half['target'] = 1

In [9]:
training_table = vstack([lmxb_first_half, hmxb_first_half])

In [10]:
training_table.write('training.fits', format='fits', overwrite=True)

* Testing set

In [11]:
lmxb_second_half = table_lmxb[lmxb_mid_index:]
hmxb_second_half = table_hmxb[hmxb_mid_index:]

In [12]:
test_table = vstack([lmxb_second_half, hmxb_second_half])

In [13]:
test_table.write('testing.fits', format='fits', overwrite=True)

In [14]:
hdul_lmxb.close()
hdul_hmxb.close()

In [15]:
hdu_list = fits.open('training.fits')
hdu_list.info()

Filename: training.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU     44   150R x 17C   [24A, D, D, D, D, D, D, J, D, D, D, D, D, K, D, D, D]   
